### **Recap**

### `Hadoop` vs. `Spark`
* <font color = 'red'>`counters`</font> vs <font color = 'green'>`accumulators`</font>
* <font color = 'red'>`map`</font> vs <font color = 'green'>`narrow transformations`</font>
* <font color = 'red'>`reduce`</font> vs <font color = 'green'>`wide(shuffle) transformations, actions`</font>
* <font color = 'red'>`file based state sharing between workers`</font> vs <font color = 'green'>`Broadcast variables (read-only:lookup table, ML (weights))`</font>
* <font color = 'red'>`read/write to files`</font> vs <font color = 'green'>`cached variables (whole dataset: iterative algorithms)`</font>
* <font color = 'red'>`eager. Many passes overe the data. write data to disk at each iteration`</font> vs <font color = 'green'>`lazy evaluation`</font>

Narrow transformations happens in parallel:
* map
* filter
* flatMap
* mapPartitions <br>

Wide trransformations from mulitple inputs: shuffle stage:
* reduceByKey
* sort
* sortByKey

Action is used to force an action. <br>

Jobs within an appl aree exe serially. one spark job for one action.

* Stages (single staged job no shuffle needed) <br>

* Wilde transfermation : require shuffle, n+1 stages \

* Spark Core consists of two APIs

* Structured API consists of DataFrames, Datasets (Scala), Spark SQL


DAG Magic
* Pipelining
* Can do interesting optimizations by delaying work
* We use the DAG to recompute on failure

Summary

* lazy evaluation
* in-memory storage
* immutabity allows spark to be easy-to-use
* fault-tolerant
* scalable
* efficient

Why Broadcast
* values are fully sent to executors <br>

Remember that eveytime rdd is created, it goes to memory
* ensure to unperesist() to release the memory <br>

Check configuration
* sc.getConf().getAll()


### **Spark SQL**

**Setting up PySpark and creating a Spark Session (since this a new Notebook)**

**Step 1**: Install Spark

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Install Java if not already installed
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Define the URL and file name for Hadoop
hadoop_url = "https://downloads.apache.org/hadoop/common/hadoop-3.4.0/hadoop-3.4.0.tar.gz"
hadoop_file = "hadoop-3.4.0.tar.gz"

# Check if the Hadoop tar.gz file already exists
if not os.path.exists(hadoop_file):
    # Download the Hadoop tar.gz file if it doesn't exist
    !wget -q $hadoop_url
else:
    print("Hadoop tar.gz file already exists. Skipping download.")

# Extract the Hadoop tar.gz file
!tar -xzf $hadoop_file

In [3]:
# Set the path to your Hadoop installation
hadoop_path = '/content/hadoop-3.4.0'
# Set environment variables
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['HADOOP_HOME'] = hadoop_path

In [4]:
# install findspark using pip
!pip install -q findspark

In [5]:
!pip3 install pyspark==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=263838f592dced0bf7c748dc503804ff4a5f3121c8f8a7d25c052ad67d40acf1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
import findspark
findspark.init()

In [7]:
# to shutdown Spark. (RUN THIS CELL AS IS)
# if Spark is not running no problem
# if Spark is running no problem also it will be shutdown
#

try:
    spark
    print(f"{sc.master} appName: {sc.appName}")
    spark.sparkContext.stop()
    del spark; del sc
    print(f"Just Shutdown Spark.")
except NameError:
    print("Spark is NOT currently running")

Spark is NOT currently running


In [8]:
# start Spark Session (RUN THIS CELL AS IS)
#Step A: Start Spark by running the following cell

from pyspark.sql import SparkSession

try:
    spark
    print("Spark is already running")
    print(f"{sc.master} appName: {sc.appName}")
except NameError:
    print('starting Spark')
    app_name = 'Day6_Spark_Notebook'
    master = "local[*]"
    spark = SparkSession\
            .builder\
            .appName(app_name)\
            .master(master)\
            .getOrCreate()
sc = spark.sparkContext

starting Spark


In [9]:
# Import SparkSession
#from pyspark.sql import SparkSession
# Create a Spark Session
#spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark


In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

## Spark DataFrame's Key Features
* Schema Information: column names and types, making data operations more meaningful and efficient.

* Lazy Evaluation: Spark builds up a plan for executing the transformations but doesn't execute them until an action is called.

* Optimized Execution: The Catalyst Optimizer in Spark analyzes and optimizes query plans to efficiently execute DataFrame operations, leveraging various optimizations such as predicate pushdown and vectorized execution.

* Integration with Spark SQL: Spark DataFrames can be created from SQL queries, and you can execute SQL queries directly on DataFrames.

* API Familiarity: Spark DataFrames provide an API that is similar to Pandas DataFrames in Python or data frames in R, making it easy for users familiar with these tools to adapt.

* Interoperability: Spark DataFrames can be created from various data sources, including CSV, JSON, Parquet, Avro, Hive tables, and external databases, among others.<br>

## Use Cases for Spark DataFrames
* Data Cleaning and Transformation: DataFrames are often used for cleaning and transforming large datasets before analysis or machine learning.
* Exploratory Data Analysis (EDA): DataFrames provide tools to quickly compute statistics and visualize data distributions.
* Data Integration: Combining data from multiple sources can be efficiently handled with Spark DataFrames.
* ETL Pipelines: Spark DataFrames are commonly used in Extract, Transform, Load (ETL) processes due to their ability to handle large-scale data transformations and movement.

In [11]:
datapath= '/content/drive/MyDrive/Big Data/books.csv'
df = spark.read.csv(datapath, header=True, inferSchema=True)

In [12]:
df.show(5)

+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+---------------+
|bookID|               title|             authors|average_rating|      isbn|    isbn13|language_code|num_pages|ratings_count|text_reviews_count|publication_date|      publisher|
+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+---------------+
|     1|Harry Potter and ...|J.K. Rowling/Mary...|          4.57| 439785960|9.78044E12|          eng|      652|      2095690|             27591|       9/16/2006|Scholastic Inc.|
|     2|Harry Potter and ...|J.K. Rowling/Mary...|          4.49| 439358078|9.78044E12|          eng|      870|      2153167|             29221|        9/1/2004|Scholastic Inc.|
|     4|Harry Potter and ...|        J.K. Rowling|          4.42| 439554896|9.78044E12|          eng|      352

In [13]:
df.printSchema()

root
 |-- bookID: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: double (nullable = true)
 |-- language_code: string (nullable = true)
 |-- num_pages: integer (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- text_reviews_count: integer (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- publisher: string (nullable = true)



In [14]:
from pyspark.sql.functions import *
# Convert the 'average_rating' column to a numeric type
#df = df.withColumn("rating", round(col("average_rating"), 1))
# Get the top 10 books by rating
df.select("title", "average_rating") \
   .orderBy(col("average_rating").desc()) \
   .limit(10) \
   .show()

+--------------------+--------------+
|               title|average_rating|
+--------------------+--------------+
| Winchester Shotguns|           5.0|
|The Goon Show  Vo...|           5.0|
|The Diamond Color...|           5.0|
|Willem de Kooning...|           5.0|
|Zone of the Ender...|           5.0|
|The Goon Show  Vo...|           5.0|
|Bulgakov's the Ma...|           5.0|
|Comoediae 1: Acha...|           5.0|
|The Complete Theo...|           5.0|
|Middlesex Borough...|           5.0|
+--------------------+--------------+



In [15]:
# Get the top 5 books by average rating and the average number of pages
(df.select("title","average_rating","num_pages")
          .groupBy("title")
          .agg(avg("average_rating"),avg("num_pages"))
          .limit(10)
          .show())

+--------------------+-------------------+--------------+
|               title|avg(average_rating)|avg(num_pages)|
+--------------------+-------------------+--------------+
|The Lord of the R...|                4.5|         914.0|
|"Myths of the Und...|               3.75|         276.0|
|The Letters of J....|               4.15|         502.0|
|That Godless Cour...|               3.59|         228.0|
|After Many a Summ...|               3.75|         368.0|
|    The Magic Finger|               3.68|          67.0|
|The Good Husband ...|               4.07|         213.0|
|The King in the W...|                3.6|         410.0|
|The Jefferson Bib...|               3.82|         103.0|
|The Secret Dreamw...|               3.64|         320.0|
+--------------------+-------------------+--------------+



In [16]:
#Find the top 5 authors by rating
df.select("authors", "average_rating").orderBy(col("average_rating").desc()).limit(5).show()

+--------------------+--------------+
|             authors|average_rating|
+--------------------+--------------+
|      Tara MacCarthy|           5.0|
|Julie Sylvester/D...|           5.0|
|Aristophanes/F.W....|           5.0|
|Middlesex Borough...|           5.0|
|          Tim Bogenn|           5.0|
+--------------------+--------------+



In [17]:
(df.select("authors", "num_pages", "average_rating")
          .groupBy("authors")
          .agg(avg("num_pages").alias("average_pages"),
               avg("average_rating").alias("averaged_rating"))
          .orderBy(col("average_pages").desc())
          .limit(10)
          .show())

+--------------------+-------------+---------------+
|             authors|average_pages|averaged_rating|
+--------------------+-------------+---------------+
|Winston S. Church...|       4736.0|           4.45|
|Marcel Proust/C.K...|       3400.0|           4.34|
|Dennis L. Kasper/...|       2751.0|           4.35|
|           Anonymous|       2198.0|            4.7|
|Laurence L. Brunt...|       2021.0|           4.29|
|Plato/John M. Coo...|       1838.0|           4.35|
|      Richard Bausch|       1776.0|           4.18|
|Plato/Edith Hamil...|       1776.0|           4.23|
|Aristotle/Jonatha...|       1757.0|           4.43|
|      Thomas Aquinas|       1716.0|           4.13|
+--------------------+-------------+---------------+



In [18]:
# Get the top 10 authors by the total number of ratings (count())
(df.select("average_rating", "authors")
   .groupBy("authors")
   .agg(count("average_rating").alias("average_rating_counts"))
   .orderBy(col("average_rating_counts").desc())
   .limit(10)
   .show())

+--------------------+---------------------+
|             authors|average_rating_counts|
+--------------------+---------------------+
|        Stephen King|                   40|
|      P.G. Wodehouse|                   40|
|    Rumiko Takahashi|                   39|
|    Orson Scott Card|                   35|
|     Agatha Christie|                   33|
|       Piers Anthony|                   30|
|     Mercedes Lackey|                   29|
|        Sandra Brown|                   29|
|        Dick Francis|                   28|
|Margaret Weis/Tra...|                   23|
+--------------------+---------------------+



### <font color = "	#b246a8">**Try to** find the average pages by top 5 rated books <font>




### YOUR CODE

In [19]:
(df.select("publication_date","num_pages", "average_rating")
   .groupBy("publication_date")
   .agg(avg("num_pages").alias("AVG_Pages"), max("average_rating").alias("Max_Rating"))
   .orderBy(col("Max_Rating").desc(),col("AVG_Pages").desc())
   .limit(5)
   .show())

+----------------+-----------------+----------+
|publication_date|        AVG_Pages|Max_Rating|
+----------------+-----------------+----------+
|       5/15/2008|            372.0|       5.0|
|       2/22/1922|            364.0|       5.0|
|       12/7/2005|            353.5|       5.0|
|      11/17/1983|            350.0|       5.0|
|        1/1/2002|340.7391304347826|       5.0|
+----------------+-----------------+----------+



In [20]:
# Inspect the DataFrame
df.describe().show()

+-------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------+------------------+------------------+------------------+----------------+--------------------+
|summary|            bookID|               title|             authors|     average_rating|                isbn|              isbn13|language_code|         num_pages|     ratings_count|text_reviews_count|publication_date|           publisher|
+-------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------+------------------+------------------+------------------+----------------+--------------------+
|  count|             11123|               11123|               11123|              11123|               11123|               11123|        11123|             11123|             11123|             11123|           11123|               11123|
|   mean|21310.856963049537|    

In [21]:
# Collect the summary statistics
summary = df.describe().toPandas()

# Transpose the DataFrame
transposed_summary = summary.T

# Display the transposed summary
transposed_summary


,0,1,2,3,4
summary,count,mean,stddev,min,max
bookID,11123,21310.856963049537,13094.727252344426,1,45641
title,11123,1830.4,152.04209943301888,said the shotgun to the head.,魔戒首部曲：魔戒現身
authors,11123,None,None,"""John """"Red"""" Shea/Mark Wahlberg""",Éric-Emmanuel Schmitt
average_rating,11123,3.934075339386829,0.35048531165127855,0.0,5.0
isbn,11123,8.858001014882138E8,1.2376761563309767E9,000100039X,9998691567
isbn13,11123,9.759880258654748E12,4.4297584886910724E11,8.987059752E9,9.79001E12
language_code,11123,None,None,ale,zho
num_pages,11123,336.4055560550211,241.15262644400408,0,6576
ratings_count,11123,17942.848062573048,112499.15347087449,0,4597666


**Note** that this function should generally be avoided except when working with small dataframes, because it pulls the entire object into memory on a single node

In [22]:
###Foce to do garbage collection
import gc

# Delete the DataFrame
del transposed_summary

# Force garbage collection
gc.collect()


65

### **Parquet File Format**

In [23]:
df.write.parquet("output.parquet")

In [24]:
df_pq = pd.read_parquet("output.parquet")
df_pq

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9.780440e+12,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.780440e+12,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.780440e+12,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9.780440e+12,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,439682584,9.780440e+12,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9.781560e+12,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,140110879,9.780140e+12,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,140131965,9.780140e+12,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,60878827,9.780060e+12,eng,434,769,139,2/27/2007,Ecco


Parquet files, being columnar storage

In [25]:
# Display the schema of the DataFrame
print("Schema:")
print(df_pq.dtypes)

# Display the first few rows of the DataFrame
print("\nFirst few rows:")
print(df_pq.head())

# Summary statistics
print("\nSummary statistics:")
print(df_pq.describe())

Schema:
bookID                  int32
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                float64
language_code          object
num_pages               int32
ratings_count           int32
text_reviews_count      int32
publication_date       object
publisher              object
dtype: object

First few rows:
   bookID                                              title  \
0       1  Harry Potter and the Half-Blood Prince (Harry ...   
1       2  Harry Potter and the Order of the Phoenix (Har...   
2       4  Harry Potter and the Chamber of Secrets (Harry...   
3       5  Harry Potter and the Prisoner of Azkaban (Harr...   
4       8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   

                      authors  average_rating        isbn        isbn13  \
0  J.K. Rowling/Mary GrandPré            4.57   439785960  9.780440e+12   
1  J.K. Rowling/Mary GrandPré            4.49   439358078  9.78044

### Catalyst Optimizer:
* Column Pruning

In [26]:
df.select("publication_date",'title').orderBy(col("publication_date"), ascending = False).show(5)

+----------------+--------------------+
|publication_date|               title|
+----------------+--------------------+
|        9/9/2010|The Postman Alway...|
|        9/9/2009|The Empty Envelop...|
|        9/9/2008|The Tale of Despe...|
|        9/9/2004|Triss (Redwall  #15)|
|        9/9/2004|The Hip Chick's G...|
+----------------+--------------------+
only showing top 5 rows



### Using Parquet File

A Parquet file is a columnar storage file format that is widely used in the Big Data ecosystem, especially with distributed data processing frameworks like Apache Hadoop, Apache Spark, and Apache Hive. It's designed to efficiently store and process large amounts of data, providing benefits such as:

Columnar Storage: Parquet stores data in a columnar format, where values from the same column are stored together. This allows for better compression and encoding techniques, as well as improved performance for columnar operations such as aggregations and filtering.

Compression: Parquet files support various compression codecs such as Snappy, Gzip, and LZO, which can significantly reduce storage requirements and improve query performance.

Schema Evolution: Parquet files include metadata that defines the schema of the data stored within them. This metadata allows for schema evolution, meaning that changes to the schema can be accommodated without needing to rewrite the entire dataset.

Cross-Platform Compatibility: Parquet files are designed to be compatible with different computing platforms and programming languages. They can be read and written by various tools and libraries across different environments.

Splittable: Parquet files are splittable, meaning that they can be divided into smaller chunks called "splits" or "blocks." This makes Parquet files suitable for parallel processing in distributed computing frameworks.

Support for Nested Data: Parquet supports nested data structures, allowing for the storage of complex data types like arrays and structs.

In [27]:
df_pq = spark.read.parquet("output.parquet")
df_pq.select("publication_date",'title').orderBy(col("publication_date"), ascending = False).show(5)

+----------------+--------------------+
|publication_date|               title|
+----------------+--------------------+
|        9/9/2010|The Postman Alway...|
|        9/9/2009|The Empty Envelop...|
|        9/9/2008|The Tale of Despe...|
|        9/9/2004|Triss (Redwall  #15)|
|        9/9/2004|The Hip Chick's G...|
+----------------+--------------------+
only showing top 5 rows



### <font color = '#b246a8'>Evaluate below approach: Dictionary vs Dataframe</font>

In [28]:
## Approach 1:

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Explore Parquet") \
    .getOrCreate()
# Read Parquet file into a DataFrame
df_pq = spark.read.parquet("output.parquet")

# Convert DataFrame to dictionary
data_dict = {}

# Extract column names
columns = df_pq.columns

# Iterate over each column
for column in columns:
    # Extract values for the column
    values = df_pq.select(column).rdd.flatMap(lambda x: x).collect()
    # Store column values in the dictionary
    data_dict[column] = values

# Display the dictionary (for demonstration purposes)
for column, values in data_dict.items():
    print(f"{column}: {values[:5]}")

# Stop SparkSession
spark.stop()




bookID: [1, 2, 4, 5, 8]
title: ['Harry Potter and the Half-Blood Prince (Harry Potter  #6)', 'Harry Potter and the Order of the Phoenix (Harry Potter  #5)', 'Harry Potter and the Chamber of Secrets (Harry Potter  #2)', 'Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)', 'Harry Potter Boxed Set  Books 1-5 (Harry Potter  #1-5)']
authors: ['J.K. Rowling/Mary GrandPré', 'J.K. Rowling/Mary GrandPré', 'J.K. Rowling', 'J.K. Rowling/Mary GrandPré', 'J.K. Rowling/Mary GrandPré']
average_rating: [4.57, 4.49, 4.42, 4.56, 4.78]
isbn: ['439785960', '439358078', '439554896', '043965548X', '439682584']
isbn13: [9780440000000.0, 9780440000000.0, 9780440000000.0, 9780440000000.0, 9780440000000.0]
language_code: ['eng', 'eng', 'eng', 'eng', 'eng']
num_pages: [652, 870, 352, 435, 2690]
ratings_count: [2095690, 2153167, 6333, 2339585, 41428]
text_reviews_count: [27591, 29221, 244, 36325, 164]
publication_date: ['9/16/2006', '9/1/2004', '11/1/2003', '5/1/2004', '9/13/2004']
publisher: ['Scholast

In [29]:
## Approach 2:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Explore Parquet") \
    .getOrCreate()


# Read Parquet file into a DataFrame
df_pq = spark.read.parquet("output.parquet")

# Display the first few rows of the DataFrame
df_pq.show(5)

# Stop SparkSession
spark.stop()

+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+---------------+
|bookID|               title|             authors|average_rating|      isbn|    isbn13|language_code|num_pages|ratings_count|text_reviews_count|publication_date|      publisher|
+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+---------------+
|     1|Harry Potter and ...|J.K. Rowling/Mary...|          4.57| 439785960|9.78044E12|          eng|      652|      2095690|             27591|       9/16/2006|Scholastic Inc.|
|     2|Harry Potter and ...|J.K. Rowling/Mary...|          4.49| 439358078|9.78044E12|          eng|      870|      2153167|             29221|        9/1/2004|Scholastic Inc.|
|     4|Harry Potter and ...|        J.K. Rowling|          4.42| 439554896|9.78044E12|          eng|      352

### Cons for libraries:

* Lack of Structure: Dictionaries do not inherently enforce a structured representation of data with predefined column names, which can make it harder to perform structured data operations.
* Performance Limitations: Dictionaries may not be as optimized for certain types of operations as DataFrame structures, especially for large-scale data processing tasks or when working with distributed computing frameworks.

### Overall Considerations:

* Data Size: For small to medium-sized datasets, either approach may be suitable. For very large datasets, DataFrames with distributed computing capabilities may offer better performance.
* Data Complexity: If your data has a simple tabular structure, DataFrames may be a more natural choice. For complex or nested data structures, dictionaries may be more flexible.
* Tooling and Ecosystem: Consider the tools and libraries you are using in your data pipeline. If you are working with PySpark or other distributed computing frameworks, DataFrames may be a better fit due to their tight integration.

# Using SQL to interface with Spark

* The temporary view is available within the Spark Session and will be automatically dropped when the session ends.
* It allows to write SQL queries directly against your DataFrame.
* It can leverage SQL to interact with these components, enabling a unified and familiar interface for data processing.

**Step 1**: Register the DataFrame as a SQL temporary view so that you can interact with it using SQL commands


In [30]:
df.createOrReplaceTempView("books")

**Step 2**: Query the temporary view using SQL

In [31]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Explore Parquet") \
    .getOrCreate()

# Read Parquet file into a DataFrame
df = spark.read.parquet("output.parquet")

# Create a temporary view from the DataFrame
df.createOrReplaceTempView("books")

# Execute SQL query to select all columns from the "books" view
sqlDF = spark.sql("SELECT * FROM books")


**Step 3**: Filter the data so that only records where the average rating is greater than 4.5<br>

Catalyst optimizer:
* if you have a filter condition in your query, the optimizer will try to apply this condition while reading the data, so only the necessary rows are fetched.


In [32]:
sqlDF.filter("average_rating > 4.5").show()

#In this case, Spark tries to push the filter condition (average_rating > 4.5) down to the file reader, reducing the amount of data read into memory.

+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+--------------------+
|bookID|               title|             authors|average_rating|      isbn|    isbn13|language_code|num_pages|ratings_count|text_reviews_count|publication_date|           publisher|
+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+--------------------+
|     1|Harry Potter and ...|J.K. Rowling/Mary...|          4.57| 439785960|9.78044E12|          eng|      652|      2095690|             27591|       9/16/2006|     Scholastic Inc.|
|     5|Harry Potter and ...|J.K. Rowling/Mary...|          4.56|043965548X|9.78044E12|          eng|      435|      2339585|             36325|        5/1/2004|     Scholastic Inc.|
|     8|Harry Potter Boxe...|J.K. Rowling/Mary...|          4.78| 439682584|9.78044E1

**Step 4**: Filter the dataset to include books that have an average rating of greater than 4.5 and the publisher is 'Ballantine Books'

In [33]:
sqlDF.filter("average_rating > 4.5").filter("publisher == 'Ballantine Books'").show()

+------+--------------------+--------------+--------------+---------+----------+-------------+---------+-------------+------------------+----------------+----------------+
|bookID|               title|       authors|average_rating|     isbn|    isbn13|language_code|num_pages|ratings_count|text_reviews_count|publication_date|       publisher|
+------+--------------------+--------------+--------------+---------+----------+-------------+---------+-------------+------------------+----------------+----------------+
|    30|J.R.R. Tolkien 4-...|J.R.R. Tolkien|          4.59|345538374|9.78035E12|          eng|     1728|       101233|              1550|       9/25/2012|Ballantine Books|
+------+--------------------+--------------+--------------+---------+----------+-------------+---------+-------------+------------------+----------------+----------------+



**Step 5**: Add a new column that depicts for each book what proportion of the overall ratings count are text review counts

In [34]:
import pyspark.sql.functions as F

sqlDF = sqlDF.withColumn("proportion_of_ratings", F.bround((sqlDF.text_reviews_count/sqlDF.ratings_count),2))
sqlDF = sqlDF.withColumn("avg_rating_decimal", sqlDF.average_rating.cast('Decimal(4,2)'))
sqlDF.show()

+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+--------------------+---------------------+------------------+
|bookID|               title|             authors|average_rating|      isbn|    isbn13|language_code|num_pages|ratings_count|text_reviews_count|publication_date|           publisher|proportion_of_ratings|avg_rating_decimal|
+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+--------------------+---------------------+------------------+
|     1|Harry Potter and ...|J.K. Rowling/Mary...|          4.57| 439785960|9.78044E12|          eng|      652|      2095690|             27591|       9/16/2006|     Scholastic Inc.|                 0.01|              4.57|
|     2|Harry Potter and ...|J.K. Rowling/Mary...|          4.49| 439358078|9.78044E12|          eng|   

# Aggregating data using GroupBy

**Step 6**: Find the lowest average_rating where the publisher if 'Broadway Books'

In [35]:
sqlDF.filter("publisher == 'Broadway Books'").groupBy().min("avg_rating_decimal").show()

+-----------------------+
|min(avg_rating_decimal)|
+-----------------------+
|                   3.25|
+-----------------------+



**Step 7**: Find the highest average_rating where the publisher if 'Broadway Books'


In [36]:
sqlDF.filter("publisher == 'Broadway Books'").groupBy().max("avg_rating_decimal").show()
#groupBy(): group all rows into one

+-----------------------+
|max(avg_rating_decimal)|
+-----------------------+
|                   4.43|
+-----------------------+



**Step 8**: Ascertain the cumulative count of ratings received for Broadway Books

In [37]:
#First convert from string type to integer
sqlDF = sqlDF.withColumn("total_rating_count", sqlDF.ratings_count.cast('Integer'))
sqlDF.filter("publisher == 'Broadway Books'").groupBy().sum("total_rating_count").show()

+-----------------------+
|sum(total_rating_count)|
+-----------------------+
|                 695164|
+-----------------------+



**Step 9:** Ascertain the average  count of ratings received for Broadway Books

In [38]:
sqlDF.filter("publisher == 'Broadway Books'").groupBy().avg("total_rating_count").show()

+-----------------------+
|avg(total_rating_count)|
+-----------------------+
|     14187.020408163266|
+-----------------------+



**Step 10**: Show average count of ratings for each publisher

In [39]:
sqlDF.groupBy("publisher").avg("total_rating_count").show()


+--------------------+-----------------------+
|           publisher|avg(total_rating_count)|
+--------------------+-----------------------+
|       The New Press|                 1693.0|
|        Chosen Books|                  536.5|
|       Digireads.com|     24628.166666666668|
|               Ember|               351258.4|
|           IVP Books|                   10.5|
|       No Exit Press|                43253.8|
|Dabel Brothers Pr...|                  276.0|
|                 DAW|     5079.1463414634145|
|      Celestial Arts|                  442.0|
|Arcadia Publishin...|                    5.0|
|Hachette Littérature|                    1.0|
|         Cleis Press|                  428.5|
|Chicago Review Press|      6315.857142857143|
|                 HQN|                 7769.0|
|World Wrestling E...|                 1154.0|
|Wayne State Unive...|                   45.0|
|           Doubleday|      8012.916666666667|
|               Verso|     1204.8666666666666|
|   Time Life

### <font color = '#b246a8'>**Evaluate below two scripts based on Catalyst Optimizer**<font>

In [40]:
sqlDF.filter(sqlDF["average_rating"] > 4.5).select("language_code").groupBy("language_code").count().show(10)
sqlDF.select("language_code").filter(sqlDF["average_rating"] > 4.5).groupBy("language_code").count().show(10)

+-------------+-----+
|language_code|count|
+-------------+-----+
|          fre|    2|
|          zho|    9|
|          eng|  151|
|          jpn|    4|
|          grc|    2|
|        en-GB|    2|
|          wel|    1|
|        en-US|   14|
|          spa|    1|
|          ger|    3|
+-------------+-----+

+-------------+-----+
|language_code|count|
+-------------+-----+
|          fre|    2|
|          zho|    9|
|          eng|  151|
|          jpn|    4|
|          grc|    2|
|        en-GB|    2|
|          wel|    1|
|        en-US|   14|
|          spa|    1|
|          ger|    3|
+-------------+-----+





### <font color = "#b246a8">**Try to make below SQL query working in Spark SQL**:<font><br>

    select publisher,count(*)
    From sqlDF
    Group by publisher
    Having count > 50
    ORDER BY count DESC

In [41]:
##### YOUR CODE####

In [42]:
result_df = sqlDF.groupBy("publisher").count().filter("count > 100").orderBy("count", ascending = False).show()

+----------------+-----+
|       publisher|count|
+----------------+-----+
|         Vintage|  318|
|   Penguin Books|  261|
|Penguin Classics|  184|
|   Mariner Books|  150|
|Ballantine Books|  144|
|   HarperCollins|  112|
|Harper Perennial|  112|
|    Pocket Books|  111|
|          Bantam|  110|
+----------------+-----+



### Aggregation

We will utilize **.agg()** method - this method lets you pass an aggregate column expression that uses any of the aggregate functions from the pyspark.sql.functions submodule.

The *pyspark.sql.functions* submodule includes a number of useful functions for performing a variety of computations like standard deviations. Note that all the aggregation functions in this submodule take the name of a column in a GroupedData table as demonstrated via the example below

**Step 11**: Compute the standard deviation of ratings count by publisher

In [43]:
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F
# Standard deviation of ratings count by publisher
sqlDF.groupBy("publisher").agg(F.stddev("ratings_count")).orderBy(F.stddev("ratings_count"), ascending = False).show()

+--------------------+---------------------+
|           publisher|stddev(ratings_count)|
+--------------------+---------------------+
|Disney Hyperion B...|    1241802.544343504|
|     Scholastic Inc.|    963197.5380474158|
|Little  Brown and...|     952459.224639278|
|      Simon Schuster|    914632.1242599855|
|HarperCollinsPubl...|    749072.9172199015|
|               Ember|    691000.7816122931|
|    Houghton Mifflin|    588231.2736483144|
|     Algonquin Books|    563081.6536129551|
|              Norton|   470158.83434484137|
|                 NAL|    457992.3958347427|
|      Back Bay Books|   400042.44932852243|
|Houghton Mifflin ...|   340970.51820230315|
|      Hyperion Books|   309891.31074566557|
|     Signet Classics|   306878.19774343196|
|     Alfred A. Knopf|   292050.45244580234|
|          Dial Press|    271033.2209338118|
|     Riverhead Books|   265556.30612197134|
|W.W. Norton & Com...|   253362.72366017065|
|Addison Wesley Pu...|    241793.3436815414|
|        P

# Joining datasets using Spark SQL

## Join Optimization:

The optimizer can re-order join operations to minimize the amount of data shuffled across the cluster. It chooses the join order based on statistics about the data size.
For instance, joining a smaller table with a larger table by broadcasting the smaller table to all nodes reduces the need for extensive shuffling. <Br>

Spark might broadcast small df to all nodes to perform the join efficiently.

In [44]:
# Create the tables that will be joined

valuesA = [('Dog',1),('Monkey',2),('Elephant',3),('Penguin',4)]
TableA = spark.createDataFrame(valuesA,['name','id'])

valuesB = [('Lizard',1),('Dog',2),('Elephant',3),('Rat',4)]
TableB = spark.createDataFrame(valuesB,['name','id'])

TableA.show()
TableB.show()

+--------+---+
|    name| id|
+--------+---+
|     Dog|  1|
|  Monkey|  2|
|Elephant|  3|
| Penguin|  4|
+--------+---+

+--------+---+
|    name| id|
+--------+---+
|  Lizard|  1|
|     Dog|  2|
|Elephant|  3|
|     Rat|  4|
+--------+---+



In [45]:
# Create table aliases

T1 = TableA.alias('T1')
T2 = TableB.alias('T2')

In [46]:
# Inner Join on name - returns matching records from the 2 tables respectively

T1.join(T2, T1.name == T2.name, 'inner').show()


+--------+---+--------+---+
|    name| id|    name| id|
+--------+---+--------+---+
|     Dog|  1|     Dog|  2|
|Elephant|  3|Elephant|  3|
+--------+---+--------+---+



In [47]:
# Left Join Example - If you want to select all records from table 1 and return data from table 2 when it matches, you choose ‘left’

T1.join(T2, T1.name == T2.name, 'left').show()

+--------+---+--------+----+
|    name| id|    name|  id|
+--------+---+--------+----+
|  Monkey|  2|    NULL|NULL|
|     Dog|  1|     Dog|   2|
|Elephant|  3|Elephant|   3|
| Penguin|  4|    NULL|NULL|
+--------+---+--------+----+



In [48]:
# Left Join - filtering out nulls

left_join = T1.join(T2, T1.name == T2.name, 'left')
left_join.filter((T2.name).isNotNull()).show()

+--------+---+--------+---+
|    name| id|    name| id|
+--------+---+--------+---+
|     Dog|  1|     Dog|  2|
|Elephant|  3|Elephant|  3|
+--------+---+--------+---+



In [49]:
# Right Join Example - If you want to select all records from table 2 and return data from table 1 when it matches, you choose ‘right’

T1.join(T2, T1.name == T2.name, 'right').show()

+--------+----+--------+---+
|    name|  id|    name| id|
+--------+----+--------+---+
|    NULL|NULL|  Lizard|  1|
|     Dog|   1|     Dog|  2|
|Elephant|   3|Elephant|  3|
|    NULL|NULL|     Rat|  4|
+--------+----+--------+---+



In [50]:
# Full Outer Join - This shows all records from the left table and all the records from the right table and nulls where the two do not match

T1.join(T2, T1.name == T2.name,how='full').show()

+--------+----+--------+----+
|    name|  id|    name|  id|
+--------+----+--------+----+
|     Dog|   1|     Dog|   2|
|Elephant|   3|Elephant|   3|
|    NULL|NULL|  Lizard|   1|
|  Monkey|   2|    NULL|NULL|
| Penguin|   4|    NULL|NULL|
|    NULL|NULL|     Rat|   4|
+--------+----+--------+----+



In [51]:
union_dfs = T1.unionAll(T2).orderBy("name").show()

+--------+---+
|    name| id|
+--------+---+
|     Dog|  1|
|     Dog|  2|
|Elephant|  3|
|Elephant|  3|
|  Lizard|  1|
|  Monkey|  2|
| Penguin|  4|
|     Rat|  4|
+--------+---+



In [52]:
union_dfs = T1.union(T2).show()

+--------+---+
|    name| id|
+--------+---+
|     Dog|  1|
|  Monkey|  2|
|Elephant|  3|
| Penguin|  4|
|  Lizard|  1|
|     Dog|  2|
|Elephant|  3|
|     Rat|  4|
+--------+---+



### Sampling

In [53]:
# Sample 10% of the data from a DataFrame using DataFrame API
sample_df = sqlDF.sample(False, 0.1, seed=42).show()

+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+--------------------+---------------------+------------------+------------------+
|bookID|               title|             authors|average_rating|      isbn|    isbn13|language_code|num_pages|ratings_count|text_reviews_count|publication_date|           publisher|proportion_of_ratings|avg_rating_decimal|total_rating_count|
+------+--------------------+--------------------+--------------+----------+----------+-------------+---------+-------------+------------------+----------------+--------------------+---------------------+------------------+------------------+
|    12|The Ultimate Hitc...|       Douglas Adams|          4.38| 517226952|9.78052E12|          eng|      815|         3628|               254|       11/1/2005|      Gramercy Books|                 0.07|              4.38|              3628|
|    25|I'm a Stranger He...

### Error Handling

In [54]:
try:
    # Execute an action that may cause an error
    sqlDF.show(3)
except Exception as e:
    # Handle the exception
    print("An error occurred:", e)


+------+--------------------+--------------------+--------------+---------+----------+-------------+---------+-------------+------------------+----------------+---------------+---------------------+------------------+------------------+
|bookID|               title|             authors|average_rating|     isbn|    isbn13|language_code|num_pages|ratings_count|text_reviews_count|publication_date|      publisher|proportion_of_ratings|avg_rating_decimal|total_rating_count|
+------+--------------------+--------------------+--------------+---------+----------+-------------+---------+-------------+------------------+----------------+---------------+---------------------+------------------+------------------+
|     1|Harry Potter and ...|J.K. Rowling/Mary...|          4.57|439785960|9.78044E12|          eng|      652|      2095690|             27591|       9/16/2006|Scholastic Inc.|                 0.01|              4.57|           2095690|
|     2|Harry Potter and ...|J.K. Rowling/Mary...|  

### <font color = "#ec39d1">***Great Job!***<font>